In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('../data/LSTM-Multivariate_pollution.csv') 
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
x_train = df[:24865]
y_train = x_train['pollution']
x_test = df[24865:31898]
y_test = x_test['pollution']
print(y_test)

In [ ]:
train_norm = x_train['pollution']

#converted into array as all the methods available are for arrays and not lists
train_norm_arr = np.asarray(train_norm)
train_norm = np.reshape(train_norm_arr, (-1, 1))

#Scaling all values between 0 and 1 so that large values don't just dominate
scaler = MinMaxScaler(feature_range=(0, 1))
train_norm = scaler.fit_transform(train_norm)
for i in range(5):
    print(train_norm[i])

In [ ]:
count = 0
for i in range(len(train_norm)):
    if train_norm[i] == 0:
        count = count +1
print('Number of null values in train_norm = ', count)

In [ ]:
train_norm = train_norm[train_norm!=0]

In [ ]:
test_norm = x_test['pollution']
test_norm_arr = np.asarray(test_norm)
test_norm = np.reshape(test_norm_arr, (-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
test_norm = scaler.fit_transform(test_norm)
for i in range(5):
    print(test_norm[i])

In [ ]:
count = 0
for i in range(len(test_norm)):
    if test_norm[i] == 0:
        count = count + 1 
print('Number of null values in test_norm = ', count)

In [ ]:
test_norm = test_norm[test_norm != 0]

In [ ]:
print(train_norm.shape)
print(test_norm.shape)

In [ ]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X),array(y)

In [ ]:
n_steps = 3
X_split_train, y_split_train = split_sequence(train_norm, n_steps)
#for i in range(len(X_split_train)):
    #print(X_split_train[i], y_split_train[i])
n_features = 1
X_split_train = X_split_train.reshape((X_split_train.shape[0], X_split_train.shape[1], n_features))

In [ ]:
X_split_test, y_split_test = split_sequence(test_norm, n_steps)
for i in range(5):
    print(X_split_test[i], y_split_test[i])
n_features = 1
X_split_test = X_split_test.reshape((X_split_test.shape[0], X_split_test.shape[1], n_features))

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))

#sgd = optimizers.SGD(lr=0.001, decay=1e-5, momentum=1.0, nesterov=False)
# sgd = optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True) #good
keras.optimizers.SGD(
    learning_rate=0.01,
    momentum=0.9,
    nesterov=True,
    weight_decay=1e-5)

#keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9)
keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
hist = model.fit(X_split_train, y_split_train, validation_data=(X_split_test, y_split_test), epochs=15, verbose = 1)

In [ ]:
yhat = model.predict(X_split_test)
for i in range(5):
    print(yhat[i])

In [ ]:
pred=[]
for i in yhat:
  pred.append(i[0])
result=pd.DataFrame({'test':y_split_test,'pred':pred})
result


In [ ]:
mse = mean_squared_error(y_split_test, yhat)
from sklearn.metrics import r2_score
print('MSE: %.5f' % mse)
print('R2 Score',r2_score(y_split_test,yhat))

In [ ]:
print(type(y_split_test),type(yhat))

In [ ]:
y_testReshaped = y_split_test.reshape(-1)
y_test_plt = pd.Series(y_testReshaped)

In [ ]:
yhatReshaped = yhat.reshape(-1)
yhatplt = pd.Series(yhatReshaped)

In [ ]:
y_test_plt.plot(legend=True,label='Actual',figsize=(12,8))
yhatplt.plot(legend=True,label='Predicted')

<hr style='height:3px'>

# Applying SARIMAX(statsmodels) for Time Series Forecasting and comparing RNN and SARIMAX
## Date: 10 oct, 2024

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df.set_index('date')

In [ ]:
df = df.drop(columns=['wnd_dir', 'wnd_spd', 'snow'])

In [ ]:
ndf = df.groupby('date').mean().reset_index()
ndf.head()

In [ ]:
print(len(ndf))

In [ ]:
ndf['date'] = pd.to_datetime(ndf['date'])
ndf.set_index('date')

In [ ]:
ndf['pollution'].plot(figsize=(12,8))

In [ ]:
from statsmodels.tsa.stattools import adfuller
def adf_test(series,title=''):
    result = adfuller(series.dropna(),autolag='AIC')
    labels = ['ADF test statistics','p-value','# lags used','# observations']
    out=pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})'] = val

    print(out.to_string())
    if result[1]<=0.05:
        print('Data is stationary')
    else:
        print('Data is non-stationary')

In [ ]:
print(adf_test(ndf['pollution']))
print(adf_test(ndf['dew']))

In [ ]:
result = seasonal_decompose(ndf['pollution'],period=10)
result.plot();

In [ ]:
auto_arima(ndf['pollution'],trace=True,seasonal=True,stationary=True,exog=[ndf['dew'],ndf['temp'],ndf['rain']],m=7).summary()

In [ ]:
ndf.set_index('date',inplace=True)

In [ ]:
train= ndf[:1795]
test = ndf[1795:]

In [ ]:
test.head()

In [ ]:
ndf.index.freq='D'
train.head()
print(len(train['pollution']))
print(len(train['dew']))
print(len(train['rain']))
print(len(train['temp']))

In [ ]:
arimaModel = SARIMAX(train['pollution'],exog=train['dew'],order=(1,1,2),seasonal_order=(2, 0, [1,2], 7)).fit()

In [ ]:
start=len(train)
end=start+len(test)-1

In [ ]:
predictions = arimaModel.predict(start,end,exog=test['dew'],typ='levels').rename('SARIMA Predictions')

In [ ]:
test['pollution'].plot(legend=True,figsize=(12,6))
predictions.plot(legend=True)